In [ ]:
import os
import glob
import tqdm
import torch
import julius
import numpy as np
import omegaconf
from mllib.src.train import main

import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt

import torch.cuda
from mllib.src.evaluate import evaluate
from mllib.src.utils import prepare_device, load_yaml
from mllib.src.distrib import get_dev_wav_clarity

In [ ]:
# Root path of clarity_challenge and evaluation dataset
root_clarity_challenge = '' # [TODO]
root_evaluation = "" # [TODO]

In [ ]:
# config = "./mllib/result/mel-rnn/20230202-145405/config.yaml"
# config = './mllib/result/mel-rnn/20230203-121042/config.yaml'

# config = "./mllib/result/dnn/20230202-142249/config.yaml"
# config = "./mllib/result/dnn/20230202-163959/config.yaml"
# config = "./mllib/result/dnn/20230202-170504/config.yaml"
# config = "./mllib/result/dnn/20230202-171624/config.yaml"
# config = "./mllib/result/dnn/20230202-185453/config.yaml"
# config = "./mllib/result/dnn/20230203-115011/config.yaml"

# config= "./mllib/result/unet/20230203-183804/config.yaml"

# config= "./mllib/result/conv-tasnet/20230203-183838/config.yaml"

# config = "./result/demucs/20230201-104202/config.yaml"
# config = "./result/wav-unet/20230201-104328/config.yaml"
# config = "./result/dcunet/20230201-104116/config.yaml"

# config= "./result/conv-tasnet/20230207-080249/config.yaml"
# config= "./result/20230209/conv-tasnet/20230207-184607/config.yaml"    # samples only including target's period, PIT
# config= "./result/20230209/conv-tasnet/20230207-185011/config.yaml"    # including all samples, PIT
# config= "./result/20230209/conv-tasnet/20230208-175200/config.yaml"    # including all samples and no PIT

# config = "./result/conv-tasnet/20230215-115103/config.yaml" 
config = "./result/conv-tasnet/20230216-113419/config.yaml"

In [ ]:
solver = main(obj_config=config, return_solver=True)

In [ ]:
model = solver.model
del solver

In [ ]:
args = load_yaml(config)
n_gpu = torch.cuda.device_count()
device = prepare_device(n_gpu, cudnn_deterministic=args.solver.cudnn_deterministic)
device = torch.device("cpu")
model = model.to(device)

In [ ]:
import soundfile as sf
# Evaluation 
# Best, haspi, hasqi
# ('S11393', 'L0146', 'S10435', 'L0109')
# Worst, haspi, hasqi
# ('S10841', 'L0146', 'S10934', 'L0138')

# scene_name= 'S10841'
# listener = 'L0146'

scene_name= 'S11393'
listener = 'L0146'

eval1 = root_evaluation + "/eval1"

target_channel = 1
wavfile = eval1 + f"/scenes/{scene_name}_mix_CH{target_channel}.wav"
wav, sr = sf.read(wavfile)
wav.shape

In [ ]:
mixture = np.transpose(wav, (1, 0))
mixture = torch.tensor(data=mixture, dtype=torch.float32, device=device)
mixture = julius.resample_frac(mixture, sr, args.dset.sample_rate)
mixture.shape

In [ ]:
ipd.Audio(mixture.flatten().numpy(), rate=args.dset.sample_rate)

In [ ]:
from mllib.src.model.types import (MULTI_SPEECH_SEPERATION_MODELS,
                MULTI_CHANNEL_SEPERATION_MODELS,
                MONARCH_SPEECH_SEPARTAION_MODELS, 
                STFT_MODELS,
                WAV_MODELS,)

nchannel, nsample = mixture.shape
# num_spk = sources.shape[1]

# mono channel to stereo for source separation models
assert args.model.audio_channels == nchannel, f"Channel between {args.dset.name} and {args.model.name} did not match..."
# assert args.model.num_spk == num_spk, f"number of speakers between {args.dset.name} and {args.model.name} did not match..."

# if args.model.name in MULTI_SPEECH_SEPERATION_MODELS:
#     assert num_spk == len(args.model.sources), f"number of speakers between {args.dset.name} and {args.model.name} did not match..."

# if not source separation models, merge batch and channels
if args.model.name in MONARCH_SPEECH_SEPARTAION_MODELS:
    mixture = torch.reshape(mixture, shape=(nchannel, 1, nsample))


In [ ]:
mixture.shape, args.model.name

In [ ]:
enhanced = evaluate(mixture=mixture[None], model=model, device=device, config=args)
enhanced = torch.squeeze(enhanced, dim=0)

In [ ]:
enhanced.shape, # sources.shape

In [ ]:
enhanced = enhanced.detach().cpu()

if args.model.name in MULTI_SPEECH_SEPERATION_MODELS:
    enhanced = enhanced[0, ...]


In [ ]:
enhanced.shape, mixture.shape, type(enhanced), type(mixture)

In [ ]:
import json
from omegaconf import OmegaConf

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from recipes.icassp_2023.MLbaseline.enhance  import enhance
from recipes.icassp_2023.MLbaseline.evaluate import get_amplified_signal, amplify_signal

In [ ]:
# Evaluation 
# Best, haspi, hasqi
# ('S11393', 'L0146', 'S10435', 'L0109')
# Worst, haspi, hasqi
# ('S10841', 'L0146', 'S10934', 'L0138')
config_clarity_challenge = OmegaConf.load("./recipes/icassp_2023/MLbaseline/config_eval.yaml")

In [ ]:
listeners_file = root_clarity_challenge+ 'clarity_CEC2_icassp2023_eval/clarity_data/metadata/listeners.eval.json'
scene_file = root_clarity_challenge+ 'clarity_CEC2_icassp2023_eval/clarity_data/metadata/listeners.eval.json'

with open(listeners_file, "r", encoding="utf-8") as fp:
    listener_audiograms = json.load(fp)
audiogram = listener_audiograms[listener]

In [ ]:
listeners_file = root_evaluation + '/metadata/listeners.eval.json'


with open(listeners_file, "r", encoding="utf-8") as fp:
    listener_audiograms = json.load(fp)
audiogram = listener_audiograms[listener]

In [ ]:
enhanced_resample = julius.resample.resample_frac(enhanced, args.dset.sample_rate, config_clarity_challenge.nalr.fs)
mixture_resample = julius.resample.resample_frac(mixture, args.dset.sample_rate, config_clarity_challenge.nalr.fs)

enhancer = NALR(**config_clarity_challenge.nalr)
compressor = Compressor(**config_clarity_challenge.compressor)

out_l = amplify_signal(enhanced_resample[0, ...], audiogram, "l", enhancer, compressor)
out_r = amplify_signal(enhanced_resample[1, ...], audiogram, "r", enhancer, compressor)
amplified = np.stack([out_l, out_r], axis=0)

if config_clarity_challenge.soft_clip:
    amplified = np.tanh(amplified)

amplified_sample_model = julius.resample.resample_frac(torch.tensor(amplified, dtype=torch.float32), 
                                                        config_clarity_challenge.nalr.fs, 
                                                        args.dset.sample_rate)

In [ ]:
audiogram, 

In [ ]:
amplified.shape

In [ ]:
enhanced_np = enhanced
mixture_np = mixture
amplified_np = amplified_sample_model 

In [ ]:
type(amplified_np), type(enhanced_np), type(mixture_np)

In [ ]:
enhanced_np = enhanced_np.numpy().flatten()
mixture_np = mixture_np.numpy().flatten()
amplified_np = amplified_np.numpy().flatten()

In [ ]:
enhanced_np.shape, mixture_np.shape, amplified_np.shape

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3)

ax0.plot(mixture_np)
ax1.plot(enhanced_np)
ax2.plot(amplified_np)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, sharey=True)

def show_stft(y, _fig, _ax):
    D = librosa.stft(y, n_fft=4096)  # STFT of y
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    img = librosa.display.specshow(S_db, ax=_ax)
    _fig.colorbar(img, ax=_ax)

show_stft(mixture_np, fig, ax0)
show_stft(enhanced_np, fig, ax1)
show_stft(amplified_np, fig, ax2)

In [ ]:
ipd.Audio(mixture_np, rate=args.dset.sample_rate)

In [ ]:
ipd.Audio(enhanced_np, rate=args.dset.sample_rate)

In [ ]:
ipd.Audio(amplified_np, rate=args.dset.sample_rate)